# Notebook: Gerar 4 submissions (log1p target)

Este notebook treina 4 modelos diferentes (RandomForest, GradientBoosting, XGBoost, LightGBM) aplicando `log1p` no target, faz pré-processamento automático (imputação e one-hot) e gera 4 arquivos `submission_*.csv`. Ajuste `CSV_PATH` se necessário.

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import xgboost as xgb
print("xgboost:", xgb.__version__)
print("pandas:  ", pd.__version__)

import lightgbm as lgb

print("Imports completos.")

In [ ]:
# === Ajuste para seus paths (JA VEM PRONTO) ===
CSV_PATH = "/Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data"

df_path = f"{CSV_PATH}/onlyfulldata_sem_outliers_cols_especificas.csv"
test_path = f"{CSV_PATH}/teste.csv"

print("Carregando arquivos:")
print("  treino ->", df_path)
print("  teste  ->", test_path)

df = pd.read_csv(df_path)
df_test = pd.read_csv(test_path)

print("Shapes:")
print("  Treino:", df.shape)
print("  Teste :", df_test.shape)

df.head()

In [ ]:
# === Preparação + log1p no target ===

# Garantir que a coluna alvo é numérica
df["annual_medical_cost"] = pd.to_numeric(df["annual_medical_cost"], errors="coerce")

# Remover registros com target inválido
df = df.dropna(subset=["annual_medical_cost"]).copy()

# Remover a coluna _income_outlier_flag se existir (treino e teste)
if "_income_outlier_flag" in df.columns:
    df = df.drop(columns=["_income_outlier_flag"])

if "_income_outlier_flag" in df_test.columns:
    df_test = df_test.drop(columns=["_income_outlier_flag"])

# Aplicar log1p no target
y = np.log1p(df["annual_medical_cost"])

# X = tudo exceto o target
X = df.drop(columns=["annual_medical_cost"])
X_test = df_test.copy()

# Separar tipos de colunas
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

print(f"Numéricas: {len(num_cols)} | Categóricas: {len(cat_cols)}")
print("Exemplos de colunas numéricas:", num_cols[:10])
print("Exemplos de colunas categóricas:", cat_cols[:10])

# Pré-processamento
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="__missing__")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

print("Pré-processador criado.")


In [ ]:
def train_and_export(model, name, X=X, y=y, X_test=df_test, preprocessor=preprocessor, out_dir='.'):
    pipe = Pipeline([
        ("prep", preprocessor),
        ("model", model)
    ])

    print(f"\nTreinando modelo: {name} ...")
    pipe.fit(X, y)

    print("Gerando previsões...")
    preds = pipe.predict(X_test)

    # desfaz log1p
    preds = np.expm1(preds)

    # proteção contra negativos / NaN
    preds = np.clip(preds, 0, None)
    preds = np.nan_to_num(preds, nan=0)

    # ⚠️ Garanta que person_id exista no df_test
    if "person_id" not in X_test.columns:
        raise KeyError("A coluna 'person_id' não existe no arquivo de teste!")

    # gerar dataframe final conforme Kaggle espera
    submission = pd.DataFrame({
        "person_id": X_test["person_id"].values,
        "annual_medical_cost": preds
    })

    out_path = os.path.join(out_dir, f"submission_{name}.csv")
    submission.to_csv(out_path, index=False)

    print(f"✔ Arquivo salvo -> {out_path}")
    return out_path


In [ ]:
# Substitua apenas o bloco que chama train_and_export para o XGBoost por este
try:
    xgb_model = xgb.XGBRegressor(
        n_estimators=1200,
        learning_rate=0.02,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist",
        n_jobs=-1,
        enable_categorical=False   # <-- correção aqui
    )
    train_and_export(xgb_model, "xgb_fixed")
except Exception as e:
    print("Falhou ao treinar XGBoost com correção. Erro:", e)




In [ ]:
# 1 — Random Forest
train_and_export(
    RandomForestRegressor(
        n_estimators=600,
        max_depth=15,
        n_jobs=-1,
        random_state=42
    ),
    "rf"
)

In [ ]:
# 2 — Gradient Boosting
train_and_export(
    GradientBoostingRegressor(
        learning_rate=0.03,
        n_estimators=800,
        max_depth=4,
        subsample=0.9,
        random_state=42
    ),
    "gb"
)

In [ ]:
 #4 — LightGBM
train_and_export(
    lgb.LGBMRegressor(
        n_estimators=1500,
        learning_rate=0.015,
        max_depth=-1,
        num_leaves=40,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1
    ),
    "lgbm"
)

print("\nTodos os modelos treinados e arquivos gerados (se possível).")

## Observações

- Se faltar alguma dependência, instale com `pip install xgboost lightgbm`.
- Ajuste `CSV_PATH` se seus arquivos estiverem em outro local.
- Se quiser que eu gere também um stacking ou faça tuning com Optuna, me avisa.